Sources: Code for sending requests to the SEC came from @ Corwien http://digtime.cn/articles/218/ai-for-trading-project-of-nlp-on-financial-statements94?order_by=created_at&

Code for the Project Helper is from Udacity @ https://github.com/udacity/artificial-intelligence-for-trading/blob/master/project/project_5/project_5_starter.ipynb

The CSV with the Loughran McDonald Dictionary is from the University of Notre Dame @ https://sraf.nd.edu/textual-analysis/resources/#Master%20Dictionary

The Watson code is from Watson @ https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#analyze


The interweaving code is done by @ Joseph Januszewicz, Josephgalaxy4@gmail.com


In [ ]:
import nltk
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import pickle
import pprint
#import project_helper
import statsmodels.api as sm
import dateutil
import re
import os
import time
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

#import project_tests

from tqdm import tqdm

In [ ]:
pip install --upgrade "ibm-watson>=5.1.0"

In [3]:
#Import Requirements
from google.colab import files
files.upload()

Saving requirements3.txt to requirements3.txt


{'requirements3.txt': b'\xef\xbb\xbfalphalens==0.3.6\r\nnltk==3.6\r\nnumpy==1.20.0\r\nratelimit==2.2.1\r\nrequests==2.18.4\r\nscikit-learn==0.24.0\r\nsix==1.11.0\r\ntqdm==4.57.0'}

In [4]:
#Import Project Helper
from google.colab import files
files.upload()

Saving project_helper.py to project_helper.py


{'project_helper.py': b'import matplotlib.pyplot as plt\nimport requests\nimport time\n\nfrom ratelimit import limits, sleep_and_retry\n\n\nclass SecAPI(object):\n    SEC_CALL_LIMIT = {\'calls\': 10, \'seconds\': 1}\n\n    @staticmethod\n    @sleep_and_retry\n    # Dividing the call limit by half to avoid coming close to the limit\n    @limits(calls=SEC_CALL_LIMIT[\'calls\'] / 2, period=SEC_CALL_LIMIT[\'seconds\'])\n    def _call_sec(url):\n        #time.sleep(0.5)\n        #print("sleep")\n        #return requests.get(url,headers={\'User-Agent\':\'josephgalaxy4@177.174.229.35.bc.googleusercontent.com\'})\n        return requests.get(url)\n\n    def get(self, url):\n        #time.sleep(10)\n        #print("sleep")\n        #return self._call_sec(url,headers={\'User-Agent\':\'josephgalaxy4@177.174.229.35.bc.googleusercontent.com\'}).text\n        return self._call_sec(url).text\n\ndef print_ten_k_data(ten_k_data, fields, field_length_limit=50):\n    indentation = \'  \'\n\n    print(\'[

In [5]:
#Import Word List from McDonald
from google.colab import files
files.upload()
sentiment_df = pd.read_csv("LoughranMcDonald_MasterDictionary_2018.csv")

Saving LoughranMcDonald_MasterDictionary_2018.csv to LoughranMcDonald_MasterDictionary_2018.csv


In [ ]:
import sys

!{sys.executable} -m pip install -r "requirements3.txt"

In [7]:
#Import Custom Files
import project_helper

In [8]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [33]:
cik_lookup = {
    "AAPL": "0000320193",
    "MSFT": "0000789019",
    'AMZN': '0001018724',
    "GOOGL": "0001652044",
    "FB": "0001326801",
    "CVX": "0000093410",
    "RDS.A": "0000089629",
    "FSLR": "0001274494",
    "NEE": "0000753308",
    "SPWR": "0000867773",
    "TSLA": "0001318605",
    "F": "0000037996",
    "NKLA": "0001731289",
    "CCIV": "0001811210",
    #"GM": "0001467858",
    #"V": "0001403161",
    #"JPM": "0000019617",
    #"BAC": "0000070858",
    #"MA": "0001141391",
    #"PYPL": "0001633917",
    #"AMT": "0001053507",
    #"CBRE": "0001138118",
    #"ZG": "0001617640",
    #"RKT": "0001805284",
    #"WELL": "0000766704",
    #"JNJ": "0000200406",
    #"AMGN": "0000318154",
    #"PFE": "0000078003",
    #"GILD": "0000882095",
    #"MRK": "0000310158",
    #"SMG" : "0000825542",
    #"IIPR": "0001677576",
    #"WEED": "0001737927",
    #"GRWG": "0001604868",
    #"CRON": "0001656472"
    }

In [34]:
ind_lookup = {
    "AAPL": "Tech",
    "MSFT": "Tech",
    'AMZN': 'Tech',
    "GOOGL": "Tech",
    "FB": "Tech",
    "CVX": "Energy",
    "RDS.A": "Energy",
    "FSLR": "Energy",
    "NEE": "Energy",
    "SPWR": "Energy",
    "TSLA": "Auto",
    "F": "Auto",
    "NKLA": "Auto",
    "CCIV": "Auto",
    "GM": "Auto",
    "V": "Finance",
    "JPM": "Finance",
    "BAC": "Finance",
    "MA": "Finance",
    "PYPL": "Finance",
    "AMT": "Real Estate",
    "CBRE": "Real Estate",
    "ZG": "Real Estate",
    "RKT": "Real Estate",
    "WELL": "Real Estate",
    "JNJ": "Pharma",
    "AMGN": "Pharma",
    "PFE": "Pharma",
    "GILD": "Pharma",
    "MRK": "Pharma",
    "SMG" : "Cananbis",
    "IIPR": "Cannabis",
    "WEED": "Cannabis",
    "GRWG": "Cannabis",
    "CRON": "Cannabis"
    }

In [35]:
sec_api = project_helper.SecAPI()

In [36]:
from bs4 import BeautifulSoup
def get_sec_data(cik, doc_type, start=0, count=2):
    newest_pricing_data = pd.to_datetime('2021-05-01')
    rss_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany' \
        '&CIK={}&type={}&start={}&count={}&owner=exclude&output=atom' \
        .format(cik, doc_type, start, count)
    i=1
    for i in range(10):
      sec_data = sec_api.get(rss_url)
      #print(sec_data)
      feed = BeautifulSoup(sec_data.encode('utf-8'), 'xml').feed
      if feed is not None:
        break
      else:
        time.sleep(1)
        #print("Sleeping!!")

    #print(feed)
    #print(feed)
    entries = [
        (
            entry.content.find('filing-href').getText(),
            entry.content.find('filing-type').getText(),
            entry.content.find('filing-date').getText())
        for entry in feed.find_all('entry', recursive=False)
        if pd.to_datetime(entry.content.find('filing-date').getText()) <= newest_pricing_data]

    return entries

In [37]:
def get_documents(text):
    """
    Extract the documents from the text

    Parameters
    ----------
    text : str
        The text with the document strings inside

    Returns
    -------
    extracted_docs : list of str
        The document strings found in `text`
    """
    
    # TODO: Implement
    final_docs = []
    start_regex = re.compile(r'<DOCUMENT>')
    end_regex = re.compile(r'</DOCUMENT>')
    
    start_idx = [x.end() for x in re.finditer(start_regex, text)]
    end_idx = [x.start() for x in re.finditer(end_regex, text)]
    
    for start_i, end_i in zip(start_idx, end_idx):
        final_docs.append(text[start_i:end_i])
    
    
    return final_docs


In [38]:
def get_document_type(doc):
    """
    Return the document type lowercased

    Parameters
    ----------
    doc : str
        The document string

    Returns
    -------
    doc_type : str
        The document type lowercased
    """
    
    # TODO: Implement
    # Regex explaination : Here I am tryng to do a positive lookbehind
    # (?<=a)b (positive lookbehind) matches the b (and only the b) in cab, but does not match bed or debt.
    # More reference : https://www.regular-expressions.info/lookaround.html
    
    type_regex = re.compile(r'(?<=<TYPE>)\w+[^\n]+') # gives out \w
    type_idx = re.search(type_regex, doc).group(0).lower()
    return type_idx


In [39]:
example_ticker = 'AMZN'
ten_ks_by_ticker = {}
#numbers={'AMZN': 0, 'BA': 0, 'CNP': 0, 'CVX': 0, 'FL': 0, 'FRT': 0, 'HON': 0}

def main_get_files(k):
  sec_data = {}
  for ticker, cik in cik_lookup.items():
    sec_data[ticker]=get_sec_data(cik, '10-Q')
  raw_fillings_by_ticker = {}
  for ticker, data in sec_data.items():
    raw_fillings_by_ticker[ticker] = {}
    for index_url, file_type, file_date in data:
        if (file_type == '10-Q'):
            file_url = index_url.replace('-index.htm', '.txt').replace('.txtl', '.txt')            
            raw_fillings_by_ticker[ticker][file_date] = sec_api.get(file_url)
  
  filling_documents_by_ticker = {}

  for ticker, raw_fillings in raw_fillings_by_ticker.items():
    filling_documents_by_ticker[ticker] = {}
    for file_date, filling in raw_fillings.items():
        filling_documents_by_ticker[ticker][file_date] = get_documents(filling)

  for ticker, filling_documents in filling_documents_by_ticker.items():
    if k==0:
      ten_ks_by_ticker[ticker] = []
    #numbers[ticker]=0
    for file_date, documents in filling_documents.items():
        for document in documents:
            #i=i+1
            if get_document_type(document) == '10-q':
                #i2=i2+1
                #numbers[ticker]=numbers[ticker]+1
                ten_ks_by_ticker[ticker].append({'CIK': cik_lookup[ticker], 'ten_q': document,'file_date': file_date})

for k in range(10):
  main_get_files(k)

In [40]:
for keys in ten_ks_by_ticker:
    print(keys," ",len(ten_ks_by_ticker[keys]))

AAPL   74
MSFT   77
AMZN   83
GOOGL   69
FB   81
CVX   58
RDS.A   77
FSLR   78
NEE   76
SPWR   74
TSLA   75
F   93
NKLA   57
CCIV   14


In [41]:
master1_ten_ks_by_ticker={}
seen_before={}

for ticks, ten_ks in ten_ks_by_ticker.items():
  master1_ten_ks_by_ticker[ticks] = []
  for line in ten_ks_by_ticker[ticks]:
    #print(line['file_date'])
    sign=line['file_date']+ticks
    if sign not in seen_before:
      #print(line['file_date'])
      seen_before[line['file_date']+ticks]=1
      master1_ten_ks_by_ticker[ticks].append(line)

In [42]:
for keys in master1_ten_ks_by_ticker:
    print(keys," ",len(master1_ten_ks_by_ticker[keys]))

AAPL   10
MSFT   10
AMZN   10
GOOGL   10
FB   10
CVX   9
RDS.A   10
FSLR   10
NEE   10
SPWR   9
TSLA   10
F   10
NKLA   8
CCIV   2


In [43]:
master_ten_ks_by_ticker=master1_ten_ks_by_ticker

In [44]:
def remove_html_tags(text):
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    return text


def clean_text(text):
    text = text.lower()
    text = remove_html_tags(text)
    
    return text

In [45]:
for ticker, ten_ks in master_ten_ks_by_ticker.items():
  for ten_k in ten_ks:
      ten_k['file_clean'] = clean_text(ten_k['ten_q'])
      #print(ten_k["file_date"])

In [46]:
def lemmatize_words(words):
    """
    Lemmatize words 

    Parameters
    ----------
    words : list of str
        List of words

    Returns
    -------
    lemmatized_words : list of str
        List of lemmatized words
    """
    
    # TODO: Implement
    wnl = WordNetLemmatizer()
    lemmatized_words = [wnl.lemmatize(word, 'v') for word in words]
    
    return lemmatized_words

In [47]:
word_pattern = re.compile('\w+')

for ticker, ten_ks in master_ten_ks_by_ticker.items():
    for ten_k in ten_ks:
        ten_k['file_lemma'] = lemmatize_words(word_pattern.findall(ten_k['file_clean']))

In [48]:
#from nltk.corpus import stopwords
lemma_english_stopwords = lemmatize_words(stopwords.words('english'))

for ticker, ten_ks in master_ten_ks_by_ticker.items():
    for ten_k in ten_ks:
        ten_k['file_lemma'] = [word for word in ten_k['file_lemma'] if word not in lemma_english_stopwords]

print('Stop Words Removed')

Stop Words Removed


In [49]:
sentiments = ['negative', 'positive', 'uncertainty', 'litigious', 'constraining', 'interesting']

#sentiment_df = pd.read_csv(os.path.join('..', '..', 'data', 'project_5_loughran_mcdonald', 'loughran_mcdonald_master_dic_2016.csv'))
sentiment_df.columns = [column.lower() for column in sentiment_df.columns] # Lowercase the columns for ease of use

# Remove unused information
sentiment_df = sentiment_df[sentiments + ['word']]
sentiment_df[sentiments] = sentiment_df[sentiments].astype(bool)
sentiment_df = sentiment_df[(sentiment_df[sentiments]).any(1)]

# Apply the same preprocessing to these words as the 10-k words
sentiment_df['word'] = lemmatize_words(sentiment_df['word'].str.lower())
sentiment_df = sentiment_df.drop_duplicates('word')

sentiment_df.head()

,negative,positive,uncertainty,litigious,constraining,interesting,word
9,True,False,False,False,False,False,abandon
12,True,False,False,False,False,False,abandonment
13,True,False,False,False,False,False,abandonments
51,True,False,False,False,False,False,abdicate
54,True,False,False,False,False,False,abdication


In [50]:
# Get dates for the universe
file_dates = {ticker: [ten_k['file_date'] for ten_k in ten_ks]
    for ticker, ten_ks in master_ten_ks_by_ticker.items()}  

In [51]:
master_dict={}
for index, row in sentiment_df.iterrows():
  master_dict[row['word']]=1


In [52]:
#q_dict={}
for ticker, ten_ks in master_ten_ks_by_ticker.items():
  for ten_k in ten_ks:
    document1=""
    for word in ten_k["file_lemma"]:
      if word in master_dict:
        document1=document1+" "+word
    ten_k["file_words"]=document1

In [53]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, SentimentOptions, EmotionOptions

In [54]:
authenticator = IAMAuthenticator('be5tVVZfmsFUh572Se2nencOEgqVCzUvZZZukhU5XB58')
natural_language_understanding = NaturalLanguageUnderstandingV1(version='2020-08-01',authenticator=authenticator)

natural_language_understanding.set_service_url('https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/8b1fe085-04ac-4040-93d5-537db24382e1')

In [55]:
company_df_dict = {'date': [], 'ticker': [], "document": [],"Industry": []}
for ticker, ten_ks in master_ten_ks_by_ticker.items():
  for ten_k in ten_ks:
    #company_df_dict["document"].append(ten_k["file_words"])
    company_df_dict["date"].append(ten_k["file_date"])
    company_df_dict["ticker"].append(ticker)
    company_df_dict["Industry"].append(ind_lookup[ticker])
    response3 = natural_language_understanding.analyze(
    text=ten_k["file_words"],
    features=Features(
        emotion=EmotionOptions(),
        sentiment=SentimentOptions())).get_result()
    company_df_dict["document"].append(response3)

company_df=pd.DataFrame(company_df_dict)
company_df.head(20)

,date,ticker,document,Industry
0,2021-04-29,AAPL,"{'usage': {'text_units': 1, 'text_characters':...",Tech
1,2020-07-31,AAPL,"{'usage': {'text_units': 1, 'text_characters':...",Tech
2,2020-05-01,AAPL,"{'usage': {'text_units': 1, 'text_characters':...",Tech
3,2019-07-31,AAPL,"{'usage': {'text_units': 2, 'text_characters':...",Tech
4,2019-05-01,AAPL,"{'usage': {'text_units': 2, 'text_characters':...",Tech
5,2019-01-30,AAPL,"{'usage': {'text_units': 2, 'text_characters':...",Tech
6,2018-08-01,AAPL,"{'usage': {'text_units': 2, 'text_characters':...",Tech
7,2018-05-02,AAPL,"{'usage': {'text_units': 2, 'text_characters':...",Tech
8,2020-01-29,AAPL,"{'usage': {'text_units': 1, 'text_characters':...",Tech
9,2021-01-28,AAPL,"{'usage': {'text_units': 1, 'text_characters':...",Tech


In [56]:
from google.colab import files
company_df.to_csv("CompSentPart1.csv")
files.download("CompSentPart1.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [58]:
files.upload()
company_df2= pd.read_csv("CompanySentimentPartTwo.csv")

Saving CompanySentimentPartTwo.csv to CompanySentimentPartTwo (1).csv


In [60]:
files.upload()
company_df3= pd.read_csv("CompSentPart3.csv")

Saving CompSentPart3.csv to CompSentPart3 (1).csv


In [63]:
frames=[company_df,company_df2,company_df3]
resultdf=pd.concat(frames)


from google.colab import files
resultdf.to_csv("FullComp.csv")
files.download("FullComp.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>